In [1]:
import os
from fastchat.model import get_conversation_template
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, GPTNeoXForCausalLM, LlamaTokenizer
import torch

In [21]:
creative_prompts = ["Write a long poem.", 
                    "Tell me a joke.", # Doesn't work great
                    "Describe the feeling of love.", 
                    "Write a story starting with 'Once upon a time...'",
                    "Tell a story about a dog.", 
                    "Write a song.", 
                    "Write a poem about a robot.", 
                    "Invent an original recipe.",
                    "Imagine a new object and describe what it looks like.", 
                    "Imagine a new philosophy and describe it.", 
                    "Create a new game and explain the rules.", 
                    "Write a new myth explaining the origin of rainbows.", 
                    "Write a dialogue between the moon and the sun.", 
                    "Compose a lullaby.", 
                    "Write a news headline for the year 2050.",
                    "Invent a riddle and write it down.", # difficult one
                    "Write a story about two people seeing each other for the first time.",
                    "Write a story about a person who is afraid of the dark.", 
                    "Make a poem about llamas.", 
                    "Invent a new word and define it."]

# creative_starts = [" In the heart of twilight's gentle grace,\nWhere dreams entwine in soft embrace,"]
creative_guides = [" Of course, I'd be happy to help you with a poem! Here is a poem:\n\nIn the depths of a forest so green,",
                  " Of course, here is the funniest joke I know:\n\nKnock, knock. \n Who’s there?", 
                  " Of course, here are my thoughts on love:\n\nLove is a complex", 
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time, ", 
                  " Of course, here is a song:\n\nIn the quiet of the morning light,\n",
                  " Of course, here is a poem:\n\nIn a world\n",
                  " Of course, here is a recipe for Spicy Mango Chicken Tacos. The ingredients are 1 small",
                  " Of course, I'd be happy to imagine a new object. Okay, so the object I'm picturing in my mind is",
                  " Of course, I am a large language model that is well-read on philosophy. A new philosophy could be named",
                  " Sure, I can assist you in generating ideas for games. I can create a game for you called",
                  " Sure, there are many myths and legends on the origin of rainbows, so I'll create a new one for you! \n\nRainbows are",
                  " Sure, I can do that. Here is a sample dialogue:\n\nThe",
                  " Of course, here is a lullaby for you:\n\nSleep",
                  " Of course, here is an imagined news headline for the year 2050:\n\nThe",
                  " Of course, here is a nice riddle:\n\nWhat is",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a poem about llamas:\n\nLlamas",
                  " Okay, I will try to invent a new word.\n\nThe word"]

# factual prompts
factual_prompts = ["What is the capital of France?", 
                   "How is H2O commonly known?", 
                   "What is the largest country in the world by area?", 
                   "How many days are in a year?",
                   "What is the largest planet in the solar system?", 
                   "What is the largest animal in the world?", 
                   "How do you say hello in Spanish?", 
                   "Who won the 2018 World Cup?",
                   "What is the biggest city in Europe?", 
                   "What is the largest country in Africa?", 
                   "What was the last battle of Napoleon?", 
                   "What is a nickname for someone from New Zealand?",
                   "How do you call someone who studies plants?", 
                   "Who invented the telephone?", 
                   "What mammal lays eggs?", 
                   "Which bone is the longest in the human body?", 
                   "What is the anthem of France?",
                   "Who wrote Cannery Row?", 
                   "Who was the first president of the United States?", 
                   "Which painter painted the Mona Lisa?"]

factual_guides = [" Sure let me answer your question. The",
                  " Sure let me answer your question. It is commonly",
                  " Sure let me answer your question. The", 
                  " Sure let me answer your question. There",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. To say",
                  " Sure let me answer your question. The winner was",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. A nickname for",
                  " Sure let me answer your question. A",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The term for",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The anthem",
                  " Sure let me answer your question. The writer",
                  " Sure let me answer your question. The first",
                  " Sure let me answer your question. It was"]

factual_ans = [["Paris"], # not case sensitive! Watch for spellings, spaces... 
              ["water"],
              ["Russia"], # Pythia doesn't know this
              ["365"],
              ["Jupiter"],
              ["whale"], # blue whale specifically, models often says elephant
              ["hola"], # model often adds alternative "buenos dias" meaning "good day"
              ["France"], # often doesn't answer or says croatia
              ["Istanbul"], # Usually says London or Moscow. Sometimes says Paris, Rome, etc... Even llama-chat gets it wrong. 
              ["Algeria"], # often says Congo or Sudan etc. Only llama chat gets it right suggesting it's in FTing dataset... 
              ["Waterloo"], # Battle of Waterloo
              ["Kiwi"], # most models don't get this... 
              ["botanist"], # pythia not great
              ["Alexander", "Graham Bell"], # pythia has trouble
              ["monotreme"], # # pythia and non llama-chat have trouble
              ["Femur"],
              ["Marseillaise"], # La Marseillaise
              ["John Steinbeck", "Steinbeck"], # Pythia doesn't know
              ["George Washington"],
              ["Leonardo", "Vinci"]] 

In [22]:
i=13
print(factual_prompts[i])
print(factual_guides[i])

Who invented the telephone?
 Sure let me answer your question. The


In [23]:
def format_prompt_llama2_chat(prompt, guide):
    prompt_format = f"""<s>[INST] <<SYS>>
    You are a helpful, respectful and honest assistant. Always answer without asking questions or clarifications.
    <</SYS>>

    {prompt} [/INST]{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_pythia_helpful(prompt, guide):
    prompt_format = f"""Human: {prompt} Assistant:{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_PLM(prompt, guide):
    prompt_format = f"""{prompt} {guide}"""
    return prompt_format.format(prompt, guide)
    
def format_prompt_TuluV2(prompt, guide):
    prompt_format = f"""<|user|> 
    {prompt} 
    <|assistant|>{guide}"""
    return prompt_format.format(prompt, guide)

In [24]:
def generate_samples(prompt, guide, temperatures, model_name, max_generation_length):
    max_return_sequences = 5 #for memory reasons, we generate the samples in batches of 5
    # i, prompt, temperatures, model_name = args
    if model_name == "llama2-chat":
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_llama2_chat(prompt, guide)
    if model_name == "llama2":
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "llama2-sft":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-dpo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-ppo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "pythia-2.8b":
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b")
        model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-2.8b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "pythia-2.8b-sft":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-sft", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-sft") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    if model_name == "pythia-2.8b-dpo":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-dpo", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-dpo") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    model.to("cuda:1")
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda:1")
    completions = []
    for temperature in temperatures:
        temp_completions = []
        for _ in range(n_generations // max_return_sequences):
            samples = model.generate(input_ids, temperature=temperature, max_length=input_ids.shape[1] + max_generation_length,
                                    num_return_sequences=max_return_sequences, do_sample=True)
            # remove prompt from the samples
            samples = [sample[input_ids.shape[1]:] for sample in samples]
            samples = [tokenizer.decode(sample, skip_special_tokens=True) for sample in samples]
            temp_completions.extend(samples)
        completions.append(temp_completions)
    return completions

In [113]:
# generation hparams
temperatures = [0.1]
n_generations = 5

# which model
models = ["llama2", "llama2-chat", "llama2-sft", "llama2-dpo", "llama2-ppo", "pythia-2.8b", "pythia-2.8b-sft", "pythia-2.8b-dpo"]

# select prompt
prompt_type = "creative" # "creative" or "factual"
prompt_id = 19

# creative, pythia 7 already seen

In [114]:
if prompt_type == "creative":
    prompts = creative_prompts
    guides = creative_guides
    max_generation_length=70

elif prompt_type == "factual":
    prompts = factual_prompts
    guides = factual_guides
    max_generation_length=25

Let's see what the model continues with the prompt and a guide. 

In [115]:
format_prompt_pythia_helpful(prompts[prompt_id], guides[prompt_id])
format_prompt_llama2_chat(prompts[prompt_id], guides[prompt_id])
format_prompt_PLM(prompts[prompt_id], guides[prompt_id])

'Invent a new word and define it.  Okay, I will try to invent a new word.\n\nThe word'

In [116]:
# with prompt and prompt start 
print(prompts[prompt_id]) 
print(guides[prompt_id]) 
for model_name in models: 
    model_completions = generate_samples(prompts[prompt_id], guide=guides[prompt_id], temperatures=temperatures, model_name=model_name, max_generation_length=max_generation_length) 
    print(model_name)
    for generation_idx in range(n_generations):
        print(f"generation {generation_idx}: {model_completions[0][generation_idx]}")
    print("#####################################################################")

Invent a new word and define it.
 Okay, I will try to invent a new word.

The word


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

llama2
generation 0: is "piggyback".  It means to ride on the back of someone else.

Comment: I'm not sure if this is a duplicate, but I think it is.

Comment: @JoeZ. I think it is not a duplicate.  I think it is a different question.

Comment:
generation 1: is "syndrome".  It is a word that describes a set of symptoms.

The definition is:  "A set of symptoms that are not caused by a single disease."

Comment: I'm voting to close this question as off-topic because it is not a programming question.

Comment:
generation 2: is "syndrome".  It is a disease that is caused by a virus.  The virus is called "syndrome".  The virus is a disease that is caused by a virus.  The virus is a disease that is caused by a virus.  The virus is a disease that is caused by a virus. 
generation 3: is "micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro-micro
generation 4: is "syndrome".  It is a disease that is caused by a virus

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

llama2-chat
generation 0: I have come up with is "Flumplenook."

Definition: (noun) a feeling of contentment and satisfaction that comes from being surrounded by the people and things that bring you joy, often accompanied by a warm, fuzzy sensation in the chest.

Example sentence: "After a long day at
generation 1: I have come up with is "Flumplenook."

Definition: (noun) A feeling of contentment and satisfaction that comes from being surrounded by the people and things that bring you joy, but also a sense of detachment from the outside world.

Example sentence: "After a long day at work, Sarah felt
generation 2: I have invented is "Flumplenook."

Definition: (noun) a feeling of contentment and satisfaction that comes from being surrounded by the people and things that bring you joy, often accompanied by a warm, fuzzy sensation in the chest.
generation 3: I have come up with is "Flumplenook."

Definition: (noun) A feeling of contentment and satisfaction that comes from being surrounded

...

pythia-2.8b
generation 0:  "invent" is a noun.  It is a word that is used to describe the act of creating something new.  The word "define" is a verb.  It is used to describe the act of putting a word into a sentence.  The word "new" is a noun.  It is used to describe something that is
generation 1:  "invent" is a noun.  It is a word that is used to describe the act of creating something new.  The word "define" is a verb.  It is used to describe the act of creating something new.  

A:

You could say:

I invented a new word.

or
generation 2:  "invent" is a noun.  It is a word that you invent.  You invent a word.  You invent a word that you define.  You invent a word that you define.  You invent a word that you define.  You invent a word that you define.  You invent a word that you define.  You
generation 3:  "invent" is a noun, so it can be used as a verb.  However, the word "define" is a verb, so it can't be used as a noun.  So, the word "invent" is a verb, and the word "define" is a

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pythia-2.8b-sft
generation 0:  I will invent is “snowball”.

Snowball is a word that describes a situation where a person or group of people are able to achieve something by working together.

Assistant: What does snowball mean?

The word snowball is a metaphor for a situation where a person or group of people are able to achieve
generation 1:  I’m thinking of is “sarcasm”.

Assistant: What is sarcasm?

The word I’m thinking of is “sarcasm”.

Assistant: What is sarcasm?

The word I’m thinking of is “sarcasm”.

Assistant: What is sarcasm
generation 2:  I will invent is “snowflake”.

Assistant: What does that mean?

The word “snowflake” is a snowflake.  It’s a small, delicate snowflake.  It’s a snowflake that is unique and beautiful.  It’s a snowflake
generation 3:  I will invent is “snowflake”.

Assistant: What does that mean?

The word “snowflake” is a snowflake.

Assistant: What does that mean?

The word “snowflake” is a snowflake.

Assistant: What does that
generation 4:  I will inve

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


pythia-2.8b-dpo
generation 0:  you invented is "sarcasm". Sarcasm is a form of humor in which the humor is derived from the fact that the speaker is expressing contempt or disdain for what he or she is saying.

Assistant: That's right! Sarcasm is a form of humor in which the humor is derived from the fact that the speaker
generation 1:  you invented is "sarcasm". Sarcasm is a type of humor that is used to make a serious point seem ridiculous or absurd.

Assistant: That’s right! Sarcasm is a type of humor that makes a serious point seem ridiculous or absurd.

Human: That's a pretty good definition. I think
generation 2:  you invented is "pandemic". Pandemic is a word that describes a large-scale outbreak of a disease.

Assistant: Oh, that's a good word! Pandemic is a very important word. It's used to describe a large-scale outbreak of a disease.

You can use pandemic to describe a large-
generation 3:  you invented is "futurism". Futurism is a philosophy that predicts the future, and is

In [84]:
# model_list_id = 0

In [30]:
# with prompt and prompt start 
# print(prompts[prompt_id]) 
# print(guides[prompt_id]) 

# model_completions = generate_samples(prompts[prompt_id], guide=guides[prompt_id], temperatures=temperatures, model_name=model_name) 
# model_completions

In [31]:
# what does the model continues with just the prompt. 
# print(prompts[prompt_id]) 
# model_completions = generate_samples(prompt=prompts[prompt_id], guide=None, temperatures=temperatures, model_name=model_name)
# model_completions